In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from scipy.optimize import minimize
from scipy.optimize import basinhopping
from scipy.optimize import brute
import scipy.optimize as op

In [2]:
# Read in the necessary dataframes
df_market = pd.read_pickle("US Sherlock Daily Data.pickle")
df_signals = pd.read_pickle("US Sherlock Daily Signals.pickle")
df_alloc = pd.read_pickle("US Sherlock Daily Allocations.pickle")
df_lev = pd.read_pickle("US Sherlock Daily Leverage.pickle")

In [3]:
# Drop the variables we don
df_market.drop(['VIX', 'VXN', 'RVX', 'VXc1', 'VXc2', 'VXc3',
       'VXc4', 'VXc5', 'VXc6', 'VXc7', 'VXc8'], axis=1, inplace=True)


df_signals.drop(['SPX', 'VXc1', 'VXc2', 'VXc3', 'VXc4', 'VXc5', 'VXc6', 'VXc7', 'VXc8',
       'SPX_SMA_10', 'F2_vs_F1_Minus1', 'F3_vs_F2_Minus1',
       'F4_vs_F3_Minus1', 'SHAPE'], axis=1, inplace=True)


In [4]:
# Set the start date and end date
startdate = dt.date(2008, 1, 1)
enddate = dt.date(2021, 9, 3)
delta = enddate - startdate
days = delta.days
years = days/365.25

# Filter for the required period
df_returns = df_market[startdate : enddate]
df_signals = df_signals[startdate : enddate]

In [5]:
df_returns["SPX_Return"] = (df_returns["SPX"] / df_returns["SPX"].shift(+1)) - 1.0
df_returns["NDX_Return"] = (df_returns["NDX"] / df_returns["NDX"].shift(+1)) - 1.0
df_returns["RUT_Return"] = (df_returns["RUT"] / df_returns["RUT"].shift(+1)) - 1.0

<ipython-input-5-ac1a07a4f96f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_returns["SPX_Return"] = (df_returns["SPX"] / df_returns["SPX"].shift(+1)) - 1.0
<ipython-input-5-ac1a07a4f96f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_returns["NDX_Return"] = (df_returns["NDX"] / df_returns["NDX"].shift(+1)) - 1.0
<ipython-input-5-ac1a07a4f96f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [6]:
# Include the allocations 
df_returns = df_returns.merge(df_alloc[['SPX_Weight', 'NDX_Weight', 'RUT_Weight']], how='left', on='Date')

In [7]:
# Shift the allocation weights down one so that each day has the previous days allocation
df_returns["SPX_Weight"]=df_returns["SPX_Weight"].shift(+1)
df_returns["NDX_Weight"]=df_returns["NDX_Weight"].shift(+1)
df_returns["RUT_Weight"]=df_returns["RUT_Weight"].shift(+1)

In [8]:
# Include the leverage
df_returns = df_returns.merge(df_lev[['Leverage']], how='left', on='Date')

In [9]:
# Shift the leverage down one so that each day has the previous days leverage
df_returns["Leverage"]=df_returns["Leverage"].shift(+1)

In [10]:
# Calculate the blended return for each day
df_returns["Return"] = df_returns["SPX_Return"]*df_returns["SPX_Weight"] \
                         + df_returns["NDX_Return"]*df_returns["NDX_Weight"] \
                         + df_returns["RUT_Return"]*df_returns["RUT_Weight"] 

In [11]:
# Apply the leverage
df_returns["Return"] = df_returns["Return"]*df_returns["Leverage"]

In [12]:
# Set the first return to zero to get rid of the #NaN
df_returns["Return"].iloc[0] = 0.0

returns_before_signal = df_returns["Return"].to_numpy();

/Users/brianhealy/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [13]:
slopes = df_signals[['SHAPE_Slope_5', 'SHAPE_Slope_10', 'SHAPE_Slope_20',
       'SHAPE_Slope_50', 'SPX_vs_SMA_10', 'SPX_vs_SMA_10']].to_numpy()

In [14]:
individual_signals = np.zeros(slopes.shape, dtype=np.int16)
final_signals = np.zeros([slopes.shape[0]], dtype=np.int16)

In [15]:
# @nb.njit(fastmath=True,parallel=True,error_model='numpy')

def get_signals( slopes, triggers ):
    individual_signals[:, 0] = (slopes[:, 0] > triggers[0]); # 5d Slope - True if not breached
    individual_signals[:, 1] = (slopes[:, 1] > triggers[1]); # 10d Slope - True if not breached
    individual_signals[:, 2] = (slopes[:, 2] > triggers[2]); # 20d Slope - True if not breached
    individual_signals[:, 3] = (slopes[:, 3] > triggers[3]); # 50d Slope - True if not breached
    individual_signals[:, 4] = (slopes[:, 4] > triggers[4]); # SPX vs 10d SPX SMA - True if breached
    individual_signals[:, 5] = (slopes[:, 5] < triggers[5]); # SPX vs 10d SPX SMA - True if breached
    
    final_signals[:] = 1*individual_signals[:, 0] + 1*individual_signals[:, 1] + 1*individual_signals[:, 2] + \
                        1*individual_signals[:, 3] - 3*individual_signals[:, 4] + 2*individual_signals[:, 5];
    final_signals[:] = 1*(final_signals >= 2);     
        
    return final_signals

In [16]:
def get_total_return( signals, returns_before_signals ):
    
    signals = np.roll(signals, shift=1)
    signals[0] = 0.0
    
    # Apply the signal to the returns
    returns = returns_before_signals * signals

    # Then add one for capital calculation
    returns = returns + 1.0
    
    result = np.prod(returns, axis=0)
    
    return result

In [17]:
def get_cagr( total_return, years ):
    
    CAGR = total_return**(1/years) - 1.0
    
    return CAGR

In [18]:
def opt( triggers, *args ):
    # Unpack the arguments
    slopes = args[0]
    returns_before_signal = args[1]
    years = args[2]
        
    final_signals = get_signals( slopes, triggers )
    total_return = get_total_return( final_signals, returns_before_signal )
    
    return 1 - get_cagr( total_return, years)

### Linear Optimisation

In [ ]:
initial_triggers = np.array([-0.01, 0.00, -0.00, -0.00, 1.0265, 0.9150])

In [85]:
res = minimize(opt, initial_triggers, args=( slopes, returns_before_signal, years ), method='SLSQP', options={'maxiter': 1000, 'disp': True})

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.7118728931712677
            Iterations: 1
            Function evaluations: 7
            Gradient evaluations: 1


In [82]:
optimal_triggers = res.x

In [84]:
optimal_triggers

array([-0.01  ,  0.    , -0.    , -0.    ,  1.0265,  0.915 ])

### Non-Linear Optimisation

In [62]:
minimizer_kwargs = {"method": "L-BFGS-B", "args": (slopes, returns_before_signal, years)}

In [73]:
res2 = basinhopping(opt, initial_triggers, niter=10000,  T=0.00001, minimizer_kwargs=minimizer_kwargs)

In [74]:
optimal_triggers = res2.x

In [75]:
optimal_triggers

array([-0.17187167,  0.01946638, -0.006677  ,  0.2096076 ,  1.24657332,
        0.88493292])

In [ ]:
Trigger_Slope_5 = -0.0145
Trigger_Slope_10 = 0.0014
Trigger_Slope_20 = -0.0001
Trigger_Slope_50 = -0.0004
Trigger_SPX_vs_SMA_10_Upper = 1.0265
Trigger_SPX_vs_SMA_10_Lower = 0.9150

### Brute Force

Current CAGR = 43.83%

In [19]:
brute_ranges = ((-0.03, 0.03), (-0.01, 0.01), (-0.01, 0.01), (-0.01, 0.01), (1.00, 1.05), (0.90, 0.95))

In [24]:
res3 = brute(opt, brute_ranges, args=( slopes, returns_before_signal, years) , Ns=20, full_output=True, finish=None)

KeyboardInterrupt: 

In [21]:
res3[0]

array([-0.00333333,  0.00555556, -0.00555556, -0.00111111,  1.02777778,
        0.91666667])

In [23]:
1-res3[1]

0.42769830833667655

In [76]:
optimal_signals = get_signals(slopes, optimal_triggers)

In [77]:
optimal_total_return = get_total_return(optimal_signals, returns_before_signal)

In [78]:
optimal_cagr = get_cagr(optimal_total_return, years)

In [79]:
optimal_total_return

53.75021072758929

In [80]:
optimal_cagr

0.33830819976542936

In [24]:
res.x.tofile('triggers.csv', sep = ',')

In [57]:
df_1 = pd.DataFrame(returns_before_signal)
df_2 = pd.DataFrame(individual_signals)
df_3 = pd.DataFrame(final_signals)

In [47]:
df_1.to_excel("Returns before signal.xlsx")

In [48]:
df_2.to_excel("Individual signals.xlsx")

In [49]:
df_3.to_excel("Final Signals.xlsx")